<b>Pandas之数据转换</b>

我们把大量的数据框非基本运算归类为“转换”运算，例如删除重复值、数值替换、分组。

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({'name':['Linda','Jacker','Linda','Alice'],'age':[12,16,12,10],'city':['A','B','A','A']},columns = ['name','age','city'])
data

,name,age,city
0,Linda,12,A
1,Jacker,16,B
2,Linda,12,A
3,Alice,10,A


<b>处理重复值</b>

DataFrame有duplicated和drop_duplicates方法  
　duplicated：返回一个布尔型 Series，将重复的行标记为 True  
　drop_duplicates：返回一个去除了重复行的新对象  
 
这两个方法默认会检查所有列，只有当两行数据的所有列都一样时，才认为是重复，可以设置subset来检查指定的列。

In [3]:
data.duplicated()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
data.duplicated(subset=['city'])

0    False
1    False
2     True
3     True
dtype: bool

In [5]:
data.drop_duplicates()

,name,age,city
0,Linda,12,A
1,Jacker,16,B
3,Alice,10,A


In [6]:
data.drop_duplicates(subset=['city'])

,name,age,city
0,Linda,12,A
1,Jacker,16,B


<b>利用映射进行转换</b>

Series 或 DataFrame 的列都可以调用一个 .map() 方法。该方法接受一个函数或字典作为参数，并将之应用于对象的每一个元素，最后返回一个包含所有结果的 Series。

具体查看第二节的“Pandas之数据对象方法”

<b>数值替换</b>

fillna方法填充缺失值可以看做值替换的一种特殊情况，Series的map方法也可以用来修改对象的数据子集，而 .replace(to_replace=None, value=None, inplace=False, limit=None, regex=False, method='pad', axis=None) 方法则提供了实现该功能的一种更简单、更灵活的方式，该方法会将to_replace的值替换为value的值。   

to_replace 参数可以是：str、 regex、 list of str, regex, or numeric、 dict, Series, or None；  
value 参数可以是：scalar, dict, list, str, regex, default None。

In [7]:
map_dict = {'A':1,'B':2}
print data.city
print data.city.map(map_dict)

0    A
1    B
2    A
3    A
Name: city, dtype: object
0    1
1    2
2    1
3    1
Name: city, dtype: int64


In [9]:
data.replace(to_replace='A',value='*')

,name,age,city
0,Linda,12,*
1,Jacker,16,B
2,Linda,12,*
3,Alice,10,*


<b>重命名轴索引</b>

DataFrame的rename(index=None,columns=None) 方法可以对index和columns进行修改。  

index 和 columns 参数并不是 index 对象，而是一个函数或字典。

In [10]:
data.rename(columns=lambda x:x+'_')

,name_,age_,city_
0,Linda,12,A
1,Jacker,16,B
2,Linda,12,A
3,Alice,10,A


In [11]:
data.index.map(lambda x:str(x)+'_')

array(['0_', '1_', '2_', '3_'], dtype=object)

In [12]:
data.columns

Index([u'name', u'age', u'city'], dtype='object')

In [13]:
data.rename(columns={data.columns[0]:'Name',data.columns[1]:'Age','city':'City'})

,Name,Age,City
0,Linda,12,A
1,Jacker,16,B
2,Linda,12,A
3,Alice,10,A


<b>指示变量</b>

在一些统计分析中（如回归模型或方差分析），分类或分组变量需要转换为指示变量列—0和1—从而建立一个决策矩阵（design matrix）。Pandas函数

pd.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False)

In [14]:
data

,name,age,city
0,Linda,12,A
1,Jacker,16,B
2,Linda,12,A
3,Alice,10,A


In [15]:
pd.get_dummies(data.name,prefix='name').join(data['city'])

,name_Alice,name_Jacker,name_Linda,city
0,0.0,0.0,1.0,A
1,0.0,1.0,0.0,B
2,0.0,0.0,1.0,A
3,1.0,0.0,0.0,A


<b>离散化</b>

为了便于分析，连续数据常常被离散化或拆分为 “面元”（bin）。这个过程要使用到 pandas 的 cut 函数：

cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)
cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)

核心参数为 x 和 bins，x 为被切对象，应当是个一维的类数组结构；bins 参数可以是序列、整数或标量

In [16]:
ser = pd.Series(np.random.randint(1,10,6))
ser

0    5
1    2
2    9
3    3
4    5
5    5
dtype: int32

bins为序列时

In [17]:
pd.cut(x=ser,bins=[0,3,6,9])

0    (3, 6]
1    (0, 3]
2    (6, 9]
3    (0, 3]
4    (3, 6]
5    (3, 6]
dtype: category
Categories (3, object): [(0, 3] < (3, 6] < (6, 9]]

bins为整数时，以 x 的上下界等长划分，可用 precision 参数调节精度

In [18]:
ser = pd.Series([2,6,7,3,8])
pd.cut(x=ser,bins=3,right=True,precision=1)

0    (2, 4]
1    (4, 6]
2    (6, 8]
3    (2, 4]
4    (6, 8]
dtype: category
Categories (3, object): [(2, 4] < (4, 6] < (6, 8]]

right=True 参数用于控制序列型 bins 的边界，默认为右包含，设为False后为左包含。labels 参数可以给离散后的结果添加新的名称。

In [19]:
ser = pd.Series([2,6,7,3,8])
pd.cut(x=ser,bins=[0,3,6,9],labels=['small','middle','large'])

0     small
1    middle
2     large
3     small
4     large
dtype: category
Categories (3, object): [small < middle < large]

In [20]:
pd.get_dummies(pd.cut(x=ser,bins=[0,3,6,9],labels=['small','middle','large']))

,small,middle,large
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


<b>抽样</b>

有时候需要对数据进行重新排序，使用numpy.random.permutation方法可以得到一个重新排序的序列。

In [21]:
data

,name,age,city
0,Linda,12,A
1,Jacker,16,B
2,Linda,12,A
3,Alice,10,A


In [22]:
order = np.random.permutation(len(data))
order

array([0, 3, 2, 1])

In [23]:
data.take(order)

,name,age,city
0,Linda,12,A
3,Alice,10,A
2,Linda,12,A
1,Jacker,16,B


使用DataFrame的sample方法可以进行随机采样。

In [24]:
data.sample(n=2)

,name,age,city
3,Alice,10,A
0,Linda,12,A
